# GitHub data integration processing - **final stage**

As part of the ** final stage ** relevant GitHub data is integrated. In notebook `03a_github_data_integration_fetch` GitHub data was fetched and stored as pickle file.

This notebooks loads the pickle file and transforms it into the final tables **github**, **github_topics**, and **github_languages**.

## Imports

In [ ]:
import pickle
import pandas as pd
from snippets import Result
from datetime import datetime, timedelta

## Load results from notebook `03a_github_data_integration_fetch`

In [ ]:
with open('data/04_consolidated/github_scrape_results.pickle', 'rb') as f:
    results= pickle.load(f)

In [ ]:
# tools where not GitHub data was retrieved
[ e["data"] for e in results if e["status"]!="okay"]

## Convert fetched GitHub data into pandas DataFrame 

In [ ]:
df=pd.DataFrame([{"id":result["id"]}|result["data"].__dict__ for result in results if result["status"]=="okay"])

In [ ]:
df.shape

### Data cleaning: drop duplicates based on GitHub name

In [ ]:
df=df.drop_duplicates(subset=["full_name"])
df.shape

### Column Type mapping

In [ ]:
df=df.astype({"name":"string",
           "full_name":"string",
"description":"string",
"homepage":"string",
"archived":"bool",
"stargazers_count":"int32",
"updated_at":"string" ,              
"created_at":"string",
"forks_count":"int32",
"subscribers_count":"int32",
"contributors_count": "int32",
"release_count":"int32",
"latest_release_name":"string",
"latest_release_date" :"string",
"license_name": "string",
"commits_count":"int32",
"latest_tag_name":"string",
"tags_count":"int32",
"latest_tag_date": "string" ,
"latest_event_date":"string",
"last_commit_date_main":"string",
"readme":"string"
           })

In [ ]:
str_with_datetime_isoformat=["updated_at","created_at","latest_release_date","latest_tag_date","latest_event_date","last_commit_date_main","request_date"]
for column in str_with_datetime_isoformat:
    df[column]=pd.to_datetime(df[column],utc=True)

In [ ]:
df.dtypes

## `github` table: save relevant columns

In [ ]:
df[['id', 'name', 'full_name', 'description', 'homepage', 'archived',
       'stargazers_count', 'updated_at', 'created_at',
       'forks_count', 'subscribers_count', 'contributors_count',
       'release_count', 'latest_release_name', 'latest_release_date',
       'license_name', 'commits_count', 'latest_tag_name', 'tags_count',
       'latest_tag_date', 'latest_event_date', 'last_commit_date_main',
       'readme', 'request_date']].to_csv("data/05_final/github.csv",index=False)

## `github_topics` table: convert and extract github topic table

In [ ]:
df_topics=df[["full_name","topics"]].explode("topics")
df_topics_notna=df_topics[df_topics["topics"].notna()]

In [ ]:
df_topics_notna.to_csv("data/05_final/github_topics.csv",index=False)


In [ ]:
df_topics_notna.drop_duplicates(subset="full_name").shape

In [ ]:
df_topics_count=df_topics.value_counts(subset="topics").reset_index(name="count")

## Convert fetched GitHub data into pandas DataFrame 

In [ ]:
df_languages=df[["full_name"]].reset_index(drop=True).join(pd.DataFrame(df["languages"].tolist()))

In [ ]:
df_languages=df_languages.melt(id_vars=["full_name"],var_name="language",value_name="count").dropna(subset="count")

In [ ]:
df_languages["percentage"]=(df_languages["count"]/df_languages.groupby('full_name')['count'].transform('sum'))*100

In [ ]:
df_languages.drop_duplicates(subset="full_name").shape

In [ ]:
df_languages.to_csv("data/05_final/github_languages.csv",index=False)